## Base Model: Neutral

In [8]:
import pandas as pd
import gurobipy as gb
from gurobipy import *

# Define the probabilities of scenarios
Ps = {
    "current": 0.5,
    "net_zero": 0.5,
}

# Define Sets
S = ["current", "net_zero"]
T = [2025, 2030, 2035]
G = ["wind", "solar", "hydro", "nuclear", "natural_gas", "geothermal", "oil", "coal"]

# Read each sheet into a DataFrame
df_Dst = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Electricity Demand')
df_Lgst = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Generation Output Targets')
df_Ugst = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Generation Capacity Targets')
df_Eg = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Emissions Factor')
df_Cgst = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Cost')
df_Bst = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Emissions Cap')
df_Kst = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Capital Expenditure Budget')

df_dev_e = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Dev_E')
df_dev_g = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Dev_G')
df_dev_q = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Dev_Q')
df_dev_c = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Dev_C')

# Convert DataFrames to dictionaries
Dst = df_Dst.set_index(['Scenario', 'Year']).to_dict()['Electricity Demand (GWh)']
Lgst = df_Lgst.set_index(['Technology', 'Scenario', 'Year']).to_dict()['Generation Output Targets (GWh)']
Ugst = df_Ugst.set_index(['Technology', 'Scenario', 'Year']).to_dict()['Generation Capacity Targets (GW)']
Cgst = df_Cgst.set_index(['Technology', 'Scenario', 'Year']).to_dict()['Cost (Millions$/GW)']
Bst = df_Bst.set_index(['Technology','Scenario', 'Year']).to_dict()['Emissions Cap (MtCO2)']
Eg = df_Eg.set_index(['Technology','Scenario','Year']).to_dict()['Emissions Factor (MtCO2/GWh)']
Kst = {scenario: df_Kst[df_Kst['Scenario'] == scenario].set_index('Year')['Value'].to_dict() for scenario in df_Kst['Scenario'].unique()}

dev_e_plus = df_dev_e.set_index(['Technology','Scenario', 'Year']).to_dict()['Deviation 5% UB']
dev_g_plus = df_dev_g.set_index(['Technology','Scenario', 'Year']).to_dict()['Deviation 5% UB']
dev_q_plus = df_dev_q.set_index(['Technology','Scenario', 'Year']).to_dict()['Deviation 5% UB']
dev_c_plus = df_dev_c.set_index(['Technology', 'Scenario','Year']).to_dict()['Deviation 5% UB']

dev_e_minus= df_dev_e.set_index(['Technology','Scenario', 'Year']).to_dict()['Deviation 5% LB']
dev_g_minus = df_dev_g.set_index(['Technology','Scenario', 'Year']).to_dict()['Deviation 5% LB']
dev_q_minus = df_dev_q.set_index(['Technology','Scenario', 'Year']).to_dict()['Deviation 5% LB']
dev_c_minus = df_dev_c.set_index(['Technology', 'Scenario','Year']).to_dict()['Deviation 5% LB']

# Create Model
m = Model("CanadaEnergyModel")

# Decision Variables
X = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, lb=0, name="Generation_Capacity")
Y = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, lb=0, name="Generation")
Z = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, lb=0, name="Emissions")
C = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, lb=0, name="Cost")
W = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, lb=0, name="Gen_Emission_Linkage")
V = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, lb=0, name="Cap_Cost_Linkage")

# Deviation Variables
et_plus = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, name="Positive_Emissions_Deviation")
et_minus = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, name="Negative_Emissions_Deviation")
ct_plus = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, name="Positive_Cost_Deviation")
ct_minus = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, name="Negative_Cost_Deviation")
gt_plus = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, name="Positive_Generation_Deviation")
gt_minus = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, name="Negative_Generation_Deviation")
qt_plus = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, name="Positive_Generation_Capacity_Deviation")
qt_minus = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, name="Negative_Generation_Capacity_Deviation")
# Set upper bounds for positive deviation variables based on data from Excel
for g in G:
    for s in S:
        for t in T:
            et_plus[g, s, t].ub = dev_e_plus.get((g, s, t), 0)
            ct_plus[g, s, t].ub = dev_c_plus.get((g, s, t), 0)
            gt_plus[g, s, t].ub = dev_g_plus.get((g, s, t), 0)
            qt_plus[g,s,t].ub = dev_q_plus.get((g,s,t), 0)


#Set upper bounds for negative deviation variables based on data from Excel
for g in G:
   for s in S:
        for t in T:
            et_minus[g,s,t].ub = dev_e_minus.get((g,s,t), 0)
            ct_minus[g,s,t].ub = dev_c_minus.get((g,s,t), 0)
            gt_minus[g,s,t].ub = dev_g_minus.get((g,s,t), 0)
            qt_minus[g,s,t].ub = dev_q_minus.get((g,s,t), 0)
            

# Constraints

# Probability sum constraint
probability_sum_constraint = m.addConstr((sum(Ps[s] for s in S) == 1), name="Probability_Sum")

# Electricity demand constraints
demand_constraints = {}
for s in S:
    for t in T:
        demand_constraints[s, t] = m.addConstr(
            sum(Y[g, s, t] for g in G) <= Dst[(s, t)],
            name=f"Demand_{s}_{t}",
        )

# Generation capacity limit constraints
capacity_limit_constraints = {}
for g in G:
    for s in S:
        for t in T:
            capacity_limit_constraints[g, s, t] = m.addConstr(
                Y[g, s, t] <= X[g, s, t] * 8760 * 1,
                name=f"Capacity_Limit_{g}_{s}_{t}",
            )

# Emission constraints
emission_constraints = {}
for s in S:
    for t in T:
        emission_constraints[s, t] = m.addConstr(
            sum(Z[g, s, t] for g in G) <= (Bst[(g,s, t)] + et_plus.sum('*', s, t) - et_minus.sum('*', s, t)),
            name=f"Emission_{g}_{s}_{t}",
        )

# Capital expenditure constraints
for s in S:
    for t in T:
        m.addConstr(
            sum(C[g, s, t] for g in G) <= Kst[s][t] + ct_plus.sum('*', s, t) - ct_minus.sum('*', s, t),
            name=f"CapEx_{s}_{t}",
        )

# Generation target constraints
for g in G:
    for s in S:
        for t in T:
            m.addConstr(
                Y[g, s, t] >= Lgst[(g, s, t)] - gt_plus[g, s, t] + gt_minus[g, s, t],
                name=f"Generation_Target_{g}_{s}_{t}",
            )

# Generation capacity target constraints
capacity_target_constraints = {}
for g in G:
    for s in S:
        for t in T:
            capacity_target_constraints[g, s, t] = m.addConstr(
                X[g, s, t] >= Ugst[(g, s, t)] - qt_plus[g, s, t] + qt_minus[g, s, t],
                name=f"Capacity_Target_{g}_{s}_{t}",
            )


# Define the emissions linking constraints dictionary
emissions_linking_constraints = {}
for g in G:
    for s in S:
        for t in T:
            emissions_linking_constraints[g, s, t] = m.addConstr(
                Z[g, s, t] == Y[g, s, t] * Eg[g, s, t],
                name=f"Emission_Linking_{g}_{s}_{t}",
            )


# Cost linking constraints
cost_linking_constraints = {}
for g in G:
    for s in S:
        for t in T:
            cost_linking_constraints[g, s, t] = m.addConstr(
                C[g, s, t] == V[g, s, t] * Cgst[(g, s, t)],
                name=f"Cost_Linking_{g}_{s}_{t}",
            )

# Emissions goal achievement constraints
for g in G:
    for s in S:
       for t in T:
            m.addConstr(
                Z[g, s, t] - et_plus[g, s, t] + et_minus[g, s, t]
               == Eg[g, s, t],
               name=f"Emissions_Goal_{g}_{s}_{t}",
            )

# Cost goal achievement constraints
for g in G:
    for s in S:
        for t in T:
            m.addConstr(
                C[g, s, t] - ct_plus[g, s, t] + ct_minus[g, s, t]
                == Bst[g,s, t],
                name=f"Cost_Goal_{g}_{s}_{t}",
            )

# Generation goal achievement constraints
for g in G:
    for s in S:
        for t in T:
            m.addConstr(
                Y[g, s, t] - gt_plus[g, s, t] + gt_minus[g, s, t]
                == Lgst[g, s, t],
                name=f"Generation_Goal_{g}_{s}_{t}",
            )

# Generation capacity goal achievement constraints
for g in G:
    for s in S:
        for t in T:
            m.addConstr(
                X[g, s, t] - qt_plus[g, s, t] + qt_minus[g, s, t]
                == Ugst[g, s, t],
                name=f"Capacity_Goal_{g}_{s}_{t}",
            )

# Define the multiple objectives
objectives = {
    "Emissions_Deviation": {"expr": LinExpr(), "priority": 3},
    "Generation_Deviation": {"expr": LinExpr(), "priority": 2},
    "Capacity_Deviation": {"expr": LinExpr(), "priority": 1},
    "Cost_Deviation": {"expr": LinExpr(), "priority": 0},
}

# Loop through scenarios
for s in S:
  # Loop through time periods
  for t in T:
    # Loop through generation technologies
    for g in G:
      # Objective expressions with g, s, t indexing
      objectives["Emissions_Deviation"]["expr"] += Ps[s] * (et_plus[g,s,t] + et_minus[g,s,t])
      objectives["Generation_Deviation"]["expr"] += Ps[s] * (gt_plus[g,s,t] + gt_minus[g,s,t])
      objectives["Capacity_Deviation"]["expr"] += Ps[s] * (qt_plus[g,s,t] + qt_minus[g,s,t])
      objectives["Cost_Deviation"]["expr"] += Ps[s] * (ct_plus[g,s,t] + ct_minus[g,s,t])

# Set the objectives using setObjectiveN
for i, (obj_name, obj_data) in enumerate(objectives.items()):
  m.setObjectiveN(obj_data["expr"], index=i, priority=obj_data["priority"], name=obj_name)

# Optimize the model
m.optimize()

# Rounding function
def round_to_2(val):
    return round(val, 2)


if m.status == GRB.OPTIMAL:
    print("Optimal Solution Found. Printing Values:")

    for s in S:
        for t in T:
            print(f"\nScenario: {s}, Time Period: {t}")
            for g in G:
                print(f"  Technology: {g}")
                generation = Y[g, s, t].x  # Use actual generation instead of capacity
                cost_per_gw = Cgst[(g, s, t)]
                total_cost = generation * cost_per_gw
                print(f"    Generation: {round_to_2(generation)} GWh")
                print(f"    Generation Capacity: {round_to_2(X[g, s, t].x)} GW")
                print(f"    Emissions: {round_to_2(Z[g, s, t].x)} MtCO2e")
                print(f"    Costs: ${round_to_2(total_cost)}")
                # Adding unrounded positive and negative deviations
                print(f"    Positive Emissions Deviation: {et_plus[g, s, t].x if et_plus[g, s, t] is not None else 0}")
                print(f"    Negative Emissions Deviation: {et_minus[g, s, t].x if et_minus[g, s, t] is not None else 0}")
                print(f"    Positive Cost Deviation: {ct_plus[g, s, t].x if ct_plus[g, s, t] is not None else 0}")
                print(f"    Negative Cost Deviation: {ct_minus[g, s, t].x if ct_minus[g, s, t] is not None else 0}")
                print(f"    Positive Generation Deviation: {gt_plus[g, s, t].x if gt_plus[g, s, t] is not None else 0}")
                print(f"    Negative Generation Deviation: {gt_minus[g, s, t].x if gt_minus[g, s, t] is not None else 0}")
                print(f"    Positive Generation Capacity Deviation: {qt_plus[g, s, t].x if qt_plus[g, s, t] is not None else 0}")
                print(f"    Negative Generation Capacity Deviation: {qt_minus[g, s, t].x if qt_minus[g, s, t] is not None else 0}")

if m.status == GRB.OPTIMAL:
    
    # Retrieve and print the values for each objective
    for i, obj_name in enumerate(objectives.keys()):
        m.setParam(GRB.Param.ObjNumber, i)
        obj_value = m.ObjNVal
        print(f"Objective '{obj_name}' Value: {obj_value}")

    # Print rounded variable values
    print("Variable Values:")
    for s in S:
        for t in T:
            for g in G:

                # Generation capacity
                rounded_X = round_to_2(X[g, s, t].x) if X[g, s, t] is not None else 0
                print(f"{X[g, s, t].varName} = {rounded_X}")

                # Generation
                rounded_Y = round_to_2(Y[g, s, t].x) if Y[g, s, t] is not None else 0
                print(f"{Y[g, s, t].varName} = {rounded_Y}")

                # Emissions
                rounded_Z = round_to_2(Z[g, s, t].x) if Z[g, s, t] is not None else 0
                print(f"{Z[g, s, t].varName} = {rounded_Z}")

                # Costs
                rounded_C = round_to_2(C[g, s, t].x) if C[g, s, t] is not None else 0
                print(f"{C[g, s, t].varName} = {rounded_C}")

                # Emission linkage
                rounded_W = round_to_2(W[g, s, t].x) if W[g, s, t] is not None else 0
                print(f"{W[g, s, t].varName} = {rounded_W}")

                # Cost linkage
                rounded_V = round_to_2(V[g, s, t].x) if V[g, s, t] is not None else 0
                print(f"{V[g, s, t].varName} = {rounded_V}")

                # Emissions deviation
                print(f"{et_plus[g, s, t].varName} = {et_plus[g, s, t].x if et_plus[g, s, t] is not None else 0}")
                print(f"{et_minus[g, s, t].varName} = {et_minus[g, s, t].x if et_minus[g, s, t] is not None else 0}")

                # Cost deviation
                print(f"{ct_plus[g, s, t].varName} = {ct_plus[g, s, t].x if ct_plus[g, s, t] is not None else 0}")

                print(f"{ct_minus[g, s, t].varName} = {ct_minus[g, s, t].x if ct_minus[g, s, t] is not None else 0}")

                # Generation deviation
                print(f"{gt_plus[g, s, t].varName} = {gt_plus[g, s, t].x if gt_plus[g, s, t] is not None else 0}")

                print(f"{gt_minus[g, s, t].varName} = {gt_minus[g, s, t].x if gt_minus[g, s, t] is not None else 0}")

                # Capacity deviation
                print(f"{qt_plus[g, s, t].varName} = {qt_plus[g, s, t].x if qt_plus[g, s, t] is not None else 0}")

                print(f"{qt_minus[g, s, t].varName} = {qt_minus[g, s, t].x if qt_minus[g, s, t] is not None else 0}")

        # Print totals for each decision variable in each scenario and time period
        for s in S:
            for t in T:
                total_gen_st = sum(Y[g, s, t].x for g in G)
                total_cap_st = sum(X[g, s, t].x for g in G)
                total_emission_st = sum(Z[g, s, t].x for g in G)
                total_cost_st = sum(C[g, s, t].x for g in G)

                print("--------------------------------------------------")
                print(f"Scenario: {s}, Time Period: {t}")
                print(f"Total Generation (GWh): {round_to_2(total_gen_st)}")
                print(f"Total Capacity (GW): {round_to_2(total_cap_st)}")
                print(f"Total Emissions (MtCO2e): {round_to_2(total_emission_st)}")
                print(f"Total Cost (Million CAD): ${round_to_2(total_cost_st)}")
                print("--------------------------------------------------\n")

        # Sum decision variables over G, S, and T dimensions for overall total
        total_gen = sum(Y[g, s, t].x for g in G for s in S for t in T)
        total_cap = sum(X[g, s, t].x for g in G for s in S for t in T)
        total_emission = sum(Z[g, s, t].x for g in G for s in S for t in T)
        total_cost = sum(C[g, s, t].x for g in G for s in S for t in T)

        print("\nOverall Total Across All Scenarios and Time Periods:")
        print(f"Total Generation (GWh): {round_to_2(total_gen)}")
        print(f"Total Capacity (GW): {round_to_2(total_cap)}")
        print(f"Total Emissions (MtCO2e): {round_to_2(total_emission)}")
        print(f"Total Cost (Million CAD): ${round_to_2(total_cost)}")

else:
    print("No optimal solution found.")


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 5700U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 451 rows, 672 columns and 1458 nonzeros
Model fingerprint: 0x3b7445fd
Variable types: 672 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e-04, 9e+03]
  Objective range  [5e-01, 5e-01]
  Bounds range     [2e+00, 5e+05]
  RHS range        [4e-04, 9e+05]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 4 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 245 rows and 254 columns
Presolved: 206 rows and 418 columns
-------------------------------------------------------

## Optimist Model

In [9]:
import pandas as pd
import gurobipy as gb
from gurobipy import *

# Define the probabilities of scenarios
Ps = {
    "current": 0.35,
    "net_zero": 0.65,
}

# Define Sets
S = ["current", "net_zero"]
T = [2025, 2030, 2035]
G = ["wind", "solar", "hydro", "nuclear", "natural_gas", "geothermal", "oil", "coal"]

# Read each sheet into a DataFrame
df_Dst = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Electricity Demand')
df_Lgst = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Generation Output Targets')
df_Ugst = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Generation Capacity Targets')
df_Eg = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Emissions Factor')
df_Cgst = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Cost')
df_Bst = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Emissions Cap')
df_Kst = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Capital Expenditure Budget')

df_dev_e = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Dev_E')
df_dev_g = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Dev_G')
df_dev_q = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Dev_Q')
df_dev_c = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Dev_C')

# Convert DataFrames to dictionaries
Dst = df_Dst.set_index(['Scenario', 'Year']).to_dict()['Electricity Demand (GWh)']
Lgst = df_Lgst.set_index(['Technology', 'Scenario', 'Year']).to_dict()['Generation Output Targets (GWh)']
Ugst = df_Ugst.set_index(['Technology', 'Scenario', 'Year']).to_dict()['Generation Capacity Targets (GW)']
Cgst = df_Cgst.set_index(['Technology', 'Scenario', 'Year']).to_dict()['Cost (Millions$/GW)']
Bst = df_Bst.set_index(['Technology','Scenario', 'Year']).to_dict()['Emissions Cap (MtCO2)']
Eg = df_Eg.set_index(['Technology','Scenario','Year']).to_dict()['Emissions Factor (MtCO2/GWh)']
Kst = {scenario: df_Kst[df_Kst['Scenario'] == scenario].set_index('Year')['Value'].to_dict() for scenario in df_Kst['Scenario'].unique()}

dev_e_plus = df_dev_e.set_index(['Technology','Scenario', 'Year']).to_dict()['Deviation 5% UB']
dev_g_plus = df_dev_g.set_index(['Technology','Scenario', 'Year']).to_dict()['Deviation 5% UB']
dev_q_plus = df_dev_q.set_index(['Technology','Scenario', 'Year']).to_dict()['Deviation 5% UB']
dev_c_plus = df_dev_c.set_index(['Technology', 'Scenario','Year']).to_dict()['Deviation 5% UB']

dev_e_minus= df_dev_e.set_index(['Technology','Scenario', 'Year']).to_dict()['Deviation 5% LB']
dev_g_minus = df_dev_g.set_index(['Technology','Scenario', 'Year']).to_dict()['Deviation 5% LB']
dev_q_minus = df_dev_q.set_index(['Technology','Scenario', 'Year']).to_dict()['Deviation 5% LB']
dev_c_minus = df_dev_c.set_index(['Technology', 'Scenario','Year']).to_dict()['Deviation 5% LB']

# Create Model
m = Model("CanadaEnergyModel")

# Decision Variables
X = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, lb=0, name="Generation_Capacity")
Y = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, lb=0, name="Generation")
Z = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, lb=0, name="Emissions")
C = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, lb=0, name="Cost")
W = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, lb=0, name="Gen_Emission_Linkage")
V = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, lb=0, name="Cap_Cost_Linkage")

# Deviation Variables
et_plus = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, name="Positive_Emissions_Deviation")
et_minus = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, name="Negative_Emissions_Deviation")
ct_plus = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, name="Positive_Cost_Deviation")
ct_minus = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, name="Negative_Cost_Deviation")
gt_plus = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, name="Positive_Generation_Deviation")
gt_minus = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, name="Negative_Generation_Deviation")
qt_plus = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, name="Positive_Generation_Capacity_Deviation")
qt_minus = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, name="Negative_Generation_Capacity_Deviation")
# Set upper bounds for positive deviation variables based on data from Excel
for g in G:
    for s in S:
        for t in T:
            et_plus[g, s, t].ub = dev_e_plus.get((g, s, t), 0)
            ct_plus[g, s, t].ub = dev_c_plus.get((g, s, t), 0)
            gt_plus[g, s, t].ub = dev_g_plus.get((g, s, t), 0)
            qt_plus[g,s,t].ub = dev_q_plus.get((g,s,t), 0)


#Set upper bounds for negative deviation variables based on data from Excel
for g in G:
   for s in S:
        for t in T:
            et_minus[g,s,t].ub = dev_e_minus.get((g,s,t), 0)
            ct_minus[g,s,t].ub = dev_c_minus.get((g,s,t), 0)
            gt_minus[g,s,t].ub = dev_g_minus.get((g,s,t), 0)
            qt_minus[g,s,t].ub = dev_q_minus.get((g,s,t), 0)
            

# Constraints

# Probability sum constraint
probability_sum_constraint = m.addConstr((sum(Ps[s] for s in S) == 1), name="Probability_Sum")

# Electricity demand constraints
demand_constraints = {}
for s in S:
    for t in T:
        demand_constraints[s, t] = m.addConstr(
            sum(Y[g, s, t] for g in G) <= Dst[(s, t)],
            name=f"Demand_{s}_{t}",
        )

# Generation capacity limit constraints
capacity_limit_constraints = {}
for g in G:
    for s in S:
        for t in T:
            capacity_limit_constraints[g, s, t] = m.addConstr(
                Y[g, s, t] <= X[g, s, t] * 8760 * 1,
                name=f"Capacity_Limit_{g}_{s}_{t}",
            )

# Emission constraints
emission_constraints = {}
for s in S:
    for t in T:
        emission_constraints[s, t] = m.addConstr(
            sum(Z[g, s, t] for g in G) <= (Bst[(g,s, t)] + et_plus.sum('*', s, t) - et_minus.sum('*', s, t)),
            name=f"Emission_{g}_{s}_{t}",
        )

# Capital expenditure constraints
for s in S:
    for t in T:
        m.addConstr(
            sum(C[g, s, t] for g in G) <= Kst[s][t] + ct_plus.sum('*', s, t) - ct_minus.sum('*', s, t),
            name=f"CapEx_{s}_{t}",
        )

# Generation target constraints
for g in G:
    for s in S:
        for t in T:
            m.addConstr(
                Y[g, s, t] >= Lgst[(g, s, t)] - gt_plus[g, s, t] + gt_minus[g, s, t],
                name=f"Generation_Target_{g}_{s}_{t}",
            )

# Generation capacity target constraints
capacity_target_constraints = {}
for g in G:
    for s in S:
        for t in T:
            capacity_target_constraints[g, s, t] = m.addConstr(
                X[g, s, t] >= Ugst[(g, s, t)] - qt_plus[g, s, t] + qt_minus[g, s, t],
                name=f"Capacity_Target_{g}_{s}_{t}",
            )


# Define the emissions linking constraints dictionary
emissions_linking_constraints = {}
for g in G:
    for s in S:
        for t in T:
            emissions_linking_constraints[g, s, t] = m.addConstr(
                Z[g, s, t] == Y[g, s, t] * Eg[g, s, t],
                name=f"Emission_Linking_{g}_{s}_{t}",
            )


# Cost linking constraints
cost_linking_constraints = {}
for g in G:
    for s in S:
        for t in T:
            cost_linking_constraints[g, s, t] = m.addConstr(
                C[g, s, t] == V[g, s, t] * Cgst[(g, s, t)],
                name=f"Cost_Linking_{g}_{s}_{t}",
            )

# Emissions goal achievement constraints
for g in G:
    for s in S:
       for t in T:
            m.addConstr(
                Z[g, s, t] - et_plus[g, s, t] + et_minus[g, s, t]
               == Eg[g, s, t],
               name=f"Emissions_Goal_{g}_{s}_{t}",
            )

# Cost goal achievement constraints
for g in G:
    for s in S:
        for t in T:
            m.addConstr(
                C[g, s, t] - ct_plus[g, s, t] + ct_minus[g, s, t]
                == Bst[g,s, t],
                name=f"Cost_Goal_{g}_{s}_{t}",
            )

# Generation goal achievement constraints
for g in G:
    for s in S:
        for t in T:
            m.addConstr(
                Y[g, s, t] - gt_plus[g, s, t] + gt_minus[g, s, t]
                == Lgst[g, s, t],
                name=f"Generation_Goal_{g}_{s}_{t}",
            )

# Generation capacity goal achievement constraints
for g in G:
    for s in S:
        for t in T:
            m.addConstr(
                X[g, s, t] - qt_plus[g, s, t] + qt_minus[g, s, t]
                == Ugst[g, s, t],
                name=f"Capacity_Goal_{g}_{s}_{t}",
            )

# Define the multiple objectives
objectives = {
    "Emissions_Deviation": {"expr": LinExpr(), "priority": 3},
    "Generation_Deviation": {"expr": LinExpr(), "priority": 2},
    "Capacity_Deviation": {"expr": LinExpr(), "priority": 1},
    "Cost_Deviation": {"expr": LinExpr(), "priority": 0},
}

# Loop through scenarios
for s in S:
  # Loop through time periods
  for t in T:
    # Loop through generation technologies
    for g in G:
      # Objective expressions with g, s, t indexing
      objectives["Emissions_Deviation"]["expr"] += Ps[s] * (et_plus[g,s,t] + et_minus[g,s,t])
      objectives["Generation_Deviation"]["expr"] += Ps[s] * (gt_plus[g,s,t] + gt_minus[g,s,t])
      objectives["Capacity_Deviation"]["expr"] += Ps[s] * (qt_plus[g,s,t] + qt_minus[g,s,t])
      objectives["Cost_Deviation"]["expr"] += Ps[s] * (ct_plus[g,s,t] + ct_minus[g,s,t])

# Set the objectives using setObjectiveN
for i, (obj_name, obj_data) in enumerate(objectives.items()):
  m.setObjectiveN(obj_data["expr"], index=i, priority=obj_data["priority"], name=obj_name)

# Optimize the model
m.optimize()

# Rounding function
def round_to_2(val):
    return round(val, 2)


if m.status == GRB.OPTIMAL:
    print("Optimal Solution Found. Printing Values:")

    for s in S:
        for t in T:
            print(f"\nScenario: {s}, Time Period: {t}")
            for g in G:
                print(f"  Technology: {g}")
                generation = Y[g, s, t].x  # Use actual generation instead of capacity
                cost_per_gw = Cgst[(g, s, t)]
                total_cost = generation * cost_per_gw
                print(f"    Generation: {round_to_2(generation)} GWh")
                print(f"    Generation Capacity: {round_to_2(X[g, s, t].x)} GW")
                print(f"    Emissions: {round_to_2(Z[g, s, t].x)} MtCO2e")
                print(f"    Costs: ${round_to_2(total_cost)}")
                # Adding unrounded positive and negative deviations
                print(f"    Positive Emissions Deviation: {et_plus[g, s, t].x if et_plus[g, s, t] is not None else 0}")
                print(f"    Negative Emissions Deviation: {et_minus[g, s, t].x if et_minus[g, s, t] is not None else 0}")
                print(f"    Positive Cost Deviation: {ct_plus[g, s, t].x if ct_plus[g, s, t] is not None else 0}")
                print(f"    Negative Cost Deviation: {ct_minus[g, s, t].x if ct_minus[g, s, t] is not None else 0}")
                print(f"    Positive Generation Deviation: {gt_plus[g, s, t].x if gt_plus[g, s, t] is not None else 0}")
                print(f"    Negative Generation Deviation: {gt_minus[g, s, t].x if gt_minus[g, s, t] is not None else 0}")
                print(f"    Positive Generation Capacity Deviation: {qt_plus[g, s, t].x if qt_plus[g, s, t] is not None else 0}")
                print(f"    Negative Generation Capacity Deviation: {qt_minus[g, s, t].x if qt_minus[g, s, t] is not None else 0}")

if m.status == GRB.OPTIMAL:
    
        # Retrieve and print the values for each objective
    for i, obj_name in enumerate(objectives.keys()):
        m.setParam(GRB.Param.ObjNumber, i)
        obj_value = m.ObjNVal
        print(f"Objective '{obj_name}' Value: {obj_value}")

    # Print rounded variable values
    print("Variable Values:")
    for s in S:
        for t in T:
            for g in G:

                # Generation capacity
                rounded_X = round_to_2(X[g, s, t].x) if X[g, s, t] is not None else 0
                print(f"{X[g, s, t].varName} = {rounded_X}")

                # Generation
                rounded_Y = round_to_2(Y[g, s, t].x) if Y[g, s, t] is not None else 0
                print(f"{Y[g, s, t].varName} = {rounded_Y}")

                # Emissions
                rounded_Z = round_to_2(Z[g, s, t].x) if Z[g, s, t] is not None else 0
                print(f"{Z[g, s, t].varName} = {rounded_Z}")

                # Costs
                rounded_C = round_to_2(C[g, s, t].x) if C[g, s, t] is not None else 0
                print(f"{C[g, s, t].varName} = {rounded_C}")

                # Emission linkage
                rounded_W = round_to_2(W[g, s, t].x) if W[g, s, t] is not None else 0
                print(f"{W[g, s, t].varName} = {rounded_W}")

                # Cost linkage
                rounded_V = round_to_2(V[g, s, t].x) if V[g, s, t] is not None else 0
                print(f"{V[g, s, t].varName} = {rounded_V}")

                # Emissions deviation
                print(f"{et_plus[g, s, t].varName} = {et_plus[g, s, t].x if et_plus[g, s, t] is not None else 0}")
                print(f"{et_minus[g, s, t].varName} = {et_minus[g, s, t].x if et_minus[g, s, t] is not None else 0}")

                # Cost deviation
                print(f"{ct_plus[g, s, t].varName} = {ct_plus[g, s, t].x if ct_plus[g, s, t] is not None else 0}")

                print(f"{ct_minus[g, s, t].varName} = {ct_minus[g, s, t].x if ct_minus[g, s, t] is not None else 0}")

                # Generation deviation
                print(f"{gt_plus[g, s, t].varName} = {gt_plus[g, s, t].x if gt_plus[g, s, t] is not None else 0}")

                print(f"{gt_minus[g, s, t].varName} = {gt_minus[g, s, t].x if gt_minus[g, s, t] is not None else 0}")

                # Capacity deviation
                print(f"{qt_plus[g, s, t].varName} = {qt_plus[g, s, t].x if qt_plus[g, s, t] is not None else 0}")

                print(f"{qt_minus[g, s, t].varName} = {qt_minus[g, s, t].x if qt_minus[g, s, t] is not None else 0}")

        # Print totals for each decision variable in each scenario and time period
        for s in S:
            for t in T:
                total_gen_st = sum(Y[g, s, t].x for g in G)
                total_cap_st = sum(X[g, s, t].x for g in G)
                total_emission_st = sum(Z[g, s, t].x for g in G)
                total_cost_st = sum(C[g, s, t].x for g in G)

                print("--------------------------------------------------")
                print(f"Scenario: {s}, Time Period: {t}")
                print(f"Total Generation (GWh): {round_to_2(total_gen_st)}")
                print(f"Total Capacity (GW): {round_to_2(total_cap_st)}")
                print(f"Total Emissions (MtCO2e): {round_to_2(total_emission_st)}")
                print(f"Total Cost (Million CAD): ${round_to_2(total_cost_st)}")
                print("--------------------------------------------------\n")

        # Sum decision variables over G, S, and T dimensions for overall total
        total_gen = sum(Y[g, s, t].x for g in G for s in S for t in T)
        total_cap = sum(X[g, s, t].x for g in G for s in S for t in T)
        total_emission = sum(Z[g, s, t].x for g in G for s in S for t in T)
        total_cost = sum(C[g, s, t].x for g in G for s in S for t in T)

        print("\nOverall Total Across All Scenarios and Time Periods:")
        print(f"Total Generation (GWh): {round_to_2(total_gen)}")
        print(f"Total Capacity (GW): {round_to_2(total_cap)}")
        print(f"Total Emissions (MtCO2e): {round_to_2(total_emission)}")
        print(f"Total Cost (Million CAD): ${round_to_2(total_cost)}")
        
else:
    print("No optimal solution found.")


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 5700U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 451 rows, 672 columns and 1458 nonzeros
Model fingerprint: 0x57a80869
Variable types: 672 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e-04, 9e+03]
  Objective range  [3e-01, 7e-01]
  Bounds range     [2e+00, 5e+05]
  RHS range        [4e-04, 9e+05]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 4 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 245 rows and 254 columns
Presolved: 206 rows and 418 columns
-------------------------------------------------------

## Pessimist Model

In [10]:
import pandas as pd
import gurobipy as gb
from gurobipy import *

# Define the probabilities of scenarios
Ps = {
    "current": 0.8,
    "net_zero": 0.2,
}

# Define Sets
S = ["current", "net_zero"]
T = [2025, 2030, 2035]
G = ["wind", "solar", "hydro", "nuclear", "natural_gas", "geothermal", "oil", "coal"]

# Read each sheet into a DataFrame
df_Dst = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Electricity Demand')
df_Lgst = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Generation Output Targets')
df_Ugst = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Generation Capacity Targets')
df_Eg = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Emissions Factor')
df_Cgst = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Cost')
df_Bst = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Emissions Cap')
df_Kst = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Capital Expenditure Budget')

df_dev_e = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Dev_E')
df_dev_g = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Dev_G')
df_dev_q = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Dev_Q')
df_dev_c = pd.read_excel(r"C:\Users\Aasna\Downloads\project data extra model (1).xlsx", sheet_name='Dev_C')

# Convert DataFrames to dictionaries
Dst = df_Dst.set_index(['Scenario', 'Year']).to_dict()['Electricity Demand (GWh)']
Lgst = df_Lgst.set_index(['Technology', 'Scenario', 'Year']).to_dict()['Generation Output Targets (GWh)']
Ugst = df_Ugst.set_index(['Technology', 'Scenario', 'Year']).to_dict()['Generation Capacity Targets (GW)']
Cgst = df_Cgst.set_index(['Technology', 'Scenario', 'Year']).to_dict()['Cost (Millions$/GW)']
Bst = df_Bst.set_index(['Technology','Scenario', 'Year']).to_dict()['Emissions Cap (MtCO2)']
Eg = df_Eg.set_index(['Technology','Scenario','Year']).to_dict()['Emissions Factor (MtCO2/GWh)']
Kst = {scenario: df_Kst[df_Kst['Scenario'] == scenario].set_index('Year')['Value'].to_dict() for scenario in df_Kst['Scenario'].unique()}

dev_e_plus = df_dev_e.set_index(['Technology','Scenario', 'Year']).to_dict()['Deviation 5% UB']
dev_g_plus = df_dev_g.set_index(['Technology','Scenario', 'Year']).to_dict()['Deviation 5% UB']
dev_q_plus = df_dev_q.set_index(['Technology','Scenario', 'Year']).to_dict()['Deviation 5% UB']
dev_c_plus = df_dev_c.set_index(['Technology', 'Scenario','Year']).to_dict()['Deviation 5% UB']

dev_e_minus= df_dev_e.set_index(['Technology','Scenario', 'Year']).to_dict()['Deviation 5% LB']
dev_g_minus = df_dev_g.set_index(['Technology','Scenario', 'Year']).to_dict()['Deviation 5% LB']
dev_q_minus = df_dev_q.set_index(['Technology','Scenario', 'Year']).to_dict()['Deviation 5% LB']
dev_c_minus = df_dev_c.set_index(['Technology', 'Scenario','Year']).to_dict()['Deviation 5% LB']

# Create Model
m = Model("CanadaEnergyModel")

# Decision Variables
X = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, lb=0, name="Generation_Capacity")
Y = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, lb=0, name="Generation")
Z = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, lb=0, name="Emissions")
C = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, lb=0, name="Cost")
W = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, lb=0, name="Gen_Emission_Linkage")
V = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, lb=0, name="Cap_Cost_Linkage")

# Deviation Variables
et_plus = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, name="Positive_Emissions_Deviation")
et_minus = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, name="Negative_Emissions_Deviation")
ct_plus = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, name="Positive_Cost_Deviation")
ct_minus = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, name="Negative_Cost_Deviation")
gt_plus = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, name="Positive_Generation_Deviation")
gt_minus = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, name="Negative_Generation_Deviation")
qt_plus = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, name="Positive_Generation_Capacity_Deviation")
qt_minus = m.addVars(G, S, T, vtype=GRB.CONTINUOUS, name="Negative_Generation_Capacity_Deviation")
# Set upper bounds for positive deviation variables based on data from Excel
for g in G:
    for s in S:
        for t in T:
            et_plus[g, s, t].ub = dev_e_plus.get((g, s, t), 0)
            ct_plus[g, s, t].ub = dev_c_plus.get((g, s, t), 0)
            gt_plus[g, s, t].ub = dev_g_plus.get((g, s, t), 0)
            qt_plus[g,s,t].ub = dev_q_plus.get((g,s,t), 0)


#Set upper bounds for negative deviation variables based on data from Excel
for g in G:
   for s in S:
        for t in T:
            et_minus[g,s,t].ub = dev_e_minus.get((g,s,t), 0)
            ct_minus[g,s,t].ub = dev_c_minus.get((g,s,t), 0)
            gt_minus[g,s,t].ub = dev_g_minus.get((g,s,t), 0)
            qt_minus[g,s,t].ub = dev_q_minus.get((g,s,t), 0)
            

# Constraints

# Probability sum constraint
probability_sum_constraint = m.addConstr((sum(Ps[s] for s in S) == 1), name="Probability_Sum")

# Electricity demand constraints
demand_constraints = {}
for s in S:
    for t in T:
        demand_constraints[s, t] = m.addConstr(
            sum(Y[g, s, t] for g in G) <= Dst[(s, t)],
            name=f"Demand_{s}_{t}",
        )

# Generation capacity limit constraints
capacity_limit_constraints = {}
for g in G:
    for s in S:
        for t in T:
            capacity_limit_constraints[g, s, t] = m.addConstr(
                Y[g, s, t] <= X[g, s, t] * 8760 * 1,
                name=f"Capacity_Limit_{g}_{s}_{t}",
            )

# Emission constraints
emission_constraints = {}
for s in S:
    for t in T:
        emission_constraints[s, t] = m.addConstr(
            sum(Z[g, s, t] for g in G) <= (Bst[(g,s, t)] + et_plus.sum('*', s, t) - et_minus.sum('*', s, t)),
            name=f"Emission_{g}_{s}_{t}",
        )

# Capital expenditure constraints
for s in S:
    for t in T:
        m.addConstr(
            sum(C[g, s, t] for g in G) <= Kst[s][t] + ct_plus.sum('*', s, t) - ct_minus.sum('*', s, t),
            name=f"CapEx_{s}_{t}",
        )

# Generation target constraints
for g in G:
    for s in S:
        for t in T:
            m.addConstr(
                Y[g, s, t] >= Lgst[(g, s, t)] - gt_plus[g, s, t] + gt_minus[g, s, t],
                name=f"Generation_Target_{g}_{s}_{t}",
            )

# Generation capacity target constraints
capacity_target_constraints = {}
for g in G:
    for s in S:
        for t in T:
            capacity_target_constraints[g, s, t] = m.addConstr(
                X[g, s, t] >= Ugst[(g, s, t)] - qt_plus[g, s, t] + qt_minus[g, s, t],
                name=f"Capacity_Target_{g}_{s}_{t}",
            )


# Define the emissions linking constraints dictionary
emissions_linking_constraints = {}
for g in G:
    for s in S:
        for t in T:
            emissions_linking_constraints[g, s, t] = m.addConstr(
                Z[g, s, t] == Y[g, s, t] * Eg[g, s, t],
                name=f"Emission_Linking_{g}_{s}_{t}",
            )


# Cost linking constraints
cost_linking_constraints = {}
for g in G:
    for s in S:
        for t in T:
            cost_linking_constraints[g, s, t] = m.addConstr(
                C[g, s, t] == V[g, s, t] * Cgst[(g, s, t)],
                name=f"Cost_Linking_{g}_{s}_{t}",
            )

# Emissions goal achievement constraints
for g in G:
    for s in S:
       for t in T:
            m.addConstr(
                Z[g, s, t] - et_plus[g, s, t] + et_minus[g, s, t]
               == Eg[g, s, t],
               name=f"Emissions_Goal_{g}_{s}_{t}",
            )

# Cost goal achievement constraints
for g in G:
    for s in S:
        for t in T:
            m.addConstr(
                C[g, s, t] - ct_plus[g, s, t] + ct_minus[g, s, t]
                == Bst[g,s, t],
                name=f"Cost_Goal_{g}_{s}_{t}",
            )

# Generation goal achievement constraints
for g in G:
    for s in S:
        for t in T:
            m.addConstr(
                Y[g, s, t] - gt_plus[g, s, t] + gt_minus[g, s, t]
                == Lgst[g, s, t],
                name=f"Generation_Goal_{g}_{s}_{t}",
            )

# Generation capacity goal achievement constraints
for g in G:
    for s in S:
        for t in T:
            m.addConstr(
                X[g, s, t] - qt_plus[g, s, t] + qt_minus[g, s, t]
                == Ugst[g, s, t],
                name=f"Capacity_Goal_{g}_{s}_{t}",
            )

# Define the multiple objectives
objectives = {
    "Emissions_Deviation": {"expr": LinExpr(), "priority": 3},
    "Generation_Deviation": {"expr": LinExpr(), "priority": 2},
    "Capacity_Deviation": {"expr": LinExpr(), "priority": 1},
    "Cost_Deviation": {"expr": LinExpr(), "priority": 0},
}

# Loop through scenarios
for s in S:
  # Loop through time periods
  for t in T:
    # Loop through generation technologies
    for g in G:
      # Objective expressions with g, s, t indexing
      objectives["Emissions_Deviation"]["expr"] += Ps[s] * (et_plus[g,s,t] + et_minus[g,s,t])
      objectives["Generation_Deviation"]["expr"] += Ps[s] * (gt_plus[g,s,t] + gt_minus[g,s,t])
      objectives["Capacity_Deviation"]["expr"] += Ps[s] * (qt_plus[g,s,t] + qt_minus[g,s,t])
      objectives["Cost_Deviation"]["expr"] += Ps[s] * (ct_plus[g,s,t] + ct_minus[g,s,t])

# Set the objectives using setObjectiveN
for i, (obj_name, obj_data) in enumerate(objectives.items()):
  m.setObjectiveN(obj_data["expr"], index=i, priority=obj_data["priority"], name=obj_name)

# Optimize the model
m.optimize()

# Rounding function
def round_to_2(val):
    return round(val, 2)


if m.status == GRB.OPTIMAL:
    print("Optimal Solution Found. Printing Values:")

    for s in S:
        for t in T:
            print(f"\nScenario: {s}, Time Period: {t}")
            for g in G:
                print(f"  Technology: {g}")
                generation = Y[g, s, t].x  # Use actual generation instead of capacity
                cost_per_gw = Cgst[(g, s, t)]
                total_cost = generation * cost_per_gw
                print(f"    Generation: {round_to_2(generation)} GWh")
                print(f"    Generation Capacity: {round_to_2(X[g, s, t].x)} GW")
                print(f"    Emissions: {round_to_2(Z[g, s, t].x)} MtCO2e")
                print(f"    Costs: ${round_to_2(total_cost)}")
                # Adding unrounded positive and negative deviations
                print(f"    Positive Emissions Deviation: {et_plus[g, s, t].x if et_plus[g, s, t] is not None else 0}")
                print(f"    Negative Emissions Deviation: {et_minus[g, s, t].x if et_minus[g, s, t] is not None else 0}")
                print(f"    Positive Cost Deviation: {ct_plus[g, s, t].x if ct_plus[g, s, t] is not None else 0}")
                print(f"    Negative Cost Deviation: {ct_minus[g, s, t].x if ct_minus[g, s, t] is not None else 0}")
                print(f"    Positive Generation Deviation: {gt_plus[g, s, t].x if gt_plus[g, s, t] is not None else 0}")
                print(f"    Negative Generation Deviation: {gt_minus[g, s, t].x if gt_minus[g, s, t] is not None else 0}")
                print(f"    Positive Generation Capacity Deviation: {qt_plus[g, s, t].x if qt_plus[g, s, t] is not None else 0}")
                print(f"    Negative Generation Capacity Deviation: {qt_minus[g, s, t].x if qt_minus[g, s, t] is not None else 0}")

if m.status == GRB.OPTIMAL:

        # Retrieve and print the values for each objective
    for i, obj_name in enumerate(objectives.keys()):
        m.setParam(GRB.Param.ObjNumber, i)
        obj_value = m.ObjNVal
        print(f"Objective '{obj_name}' Value: {obj_value}")
        
    # Print rounded variable values
    print("Variable Values:")
    for s in S:
        for t in T:
            for g in G:

                # Generation capacity
                rounded_X = round_to_2(X[g, s, t].x) if X[g, s, t] is not None else 0
                print(f"{X[g, s, t].varName} = {rounded_X}")

                # Generation
                rounded_Y = round_to_2(Y[g, s, t].x) if Y[g, s, t] is not None else 0
                print(f"{Y[g, s, t].varName} = {rounded_Y}")

                # Emissions
                rounded_Z = round_to_2(Z[g, s, t].x) if Z[g, s, t] is not None else 0
                print(f"{Z[g, s, t].varName} = {rounded_Z}")

                # Costs
                rounded_C = round_to_2(C[g, s, t].x) if C[g, s, t] is not None else 0
                print(f"{C[g, s, t].varName} = {rounded_C}")

                # Emission linkage
                rounded_W = round_to_2(W[g, s, t].x) if W[g, s, t] is not None else 0
                print(f"{W[g, s, t].varName} = {rounded_W}")

                # Cost linkage
                rounded_V = round_to_2(V[g, s, t].x) if V[g, s, t] is not None else 0
                print(f"{V[g, s, t].varName} = {rounded_V}")

                # Emissions deviation
                print(f"{et_plus[g, s, t].varName} = {et_plus[g, s, t].x if et_plus[g, s, t] is not None else 0}")
                print(f"{et_minus[g, s, t].varName} = {et_minus[g, s, t].x if et_minus[g, s, t] is not None else 0}")

                # Cost deviation
                print(f"{ct_plus[g, s, t].varName} = {ct_plus[g, s, t].x if ct_plus[g, s, t] is not None else 0}")

                print(f"{ct_minus[g, s, t].varName} = {ct_minus[g, s, t].x if ct_minus[g, s, t] is not None else 0}")

                # Generation deviation
                print(f"{gt_plus[g, s, t].varName} = {gt_plus[g, s, t].x if gt_plus[g, s, t] is not None else 0}")

                print(f"{gt_minus[g, s, t].varName} = {gt_minus[g, s, t].x if gt_minus[g, s, t] is not None else 0}")

                # Capacity deviation
                print(f"{qt_plus[g, s, t].varName} = {qt_plus[g, s, t].x if qt_plus[g, s, t] is not None else 0}")

                print(f"{qt_minus[g, s, t].varName} = {qt_minus[g, s, t].x if qt_minus[g, s, t] is not None else 0}")

        # Print totals for each decision variable in each scenario and time period
        for s in S:
            for t in T:
                total_gen_st = sum(Y[g, s, t].x for g in G)
                total_cap_st = sum(X[g, s, t].x for g in G)
                total_emission_st = sum(Z[g, s, t].x for g in G)
                total_cost_st = sum(C[g, s, t].x for g in G)

                print("--------------------------------------------------")
                print(f"Scenario: {s}, Time Period: {t}")
                print(f"Total Generation (GWh): {round_to_2(total_gen_st)}")
                print(f"Total Capacity (GW): {round_to_2(total_cap_st)}")
                print(f"Total Emissions (MtCO2e): {round_to_2(total_emission_st)}")
                print(f"Total Cost (Million CAD): ${round_to_2(total_cost_st)}")
                print("--------------------------------------------------\n")

        # Sum decision variables over G, S, and T dimensions for overall total
        total_gen = sum(Y[g, s, t].x for g in G for s in S for t in T)
        total_cap = sum(X[g, s, t].x for g in G for s in S for t in T)
        total_emission = sum(Z[g, s, t].x for g in G for s in S for t in T)
        total_cost = sum(C[g, s, t].x for g in G for s in S for t in T)

        print("\nOverall Total Across All Scenarios and Time Periods:")
        print(f"Total Generation (GWh): {round_to_2(total_gen)}")
        print(f"Total Capacity (GW): {round_to_2(total_cap)}")
        print(f"Total Emissions (MtCO2e): {round_to_2(total_emission)}")
        print(f"Total Cost (Million CAD): ${round_to_2(total_cost)}")

else:
    print("No optimal solution found.")


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 5700U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 451 rows, 672 columns and 1458 nonzeros
Model fingerprint: 0xc6cbe628
Variable types: 672 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e-04, 9e+03]
  Objective range  [2e-01, 8e-01]
  Bounds range     [2e+00, 5e+05]
  RHS range        [4e-04, 9e+05]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 4 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 245 rows and 254 columns
Presolved: 206 rows and 418 columns
-------------------------------------------------------